In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
data = pd.read_csv('NBA_Players.csv') #LER OS DADOS
data.columns = [columns.replace(' ', '') for columns in data.columns] #REMOVENDO ESPAÇO DOS NOMES DAS COLUNAS
data.drop(columns=['URL','THM_THA','FTM_FTA','FGM_FGA'], inplace=True) #REMOVENDO URL E DADOS COMPOSTOS
data.drop(data.loc[data['SALARY'] == 'Not signed'].index, inplace=True) #REMOVENDO JOGADORES COM SALÁRIO NÃO DIVULGADO
data.dropna(inplace=True) #REMOVENDO LINHAS COM DADOS VAZIOS

data['SALARY']=data['SALARY'].str.replace(',','') #REMOVEDO VÍRGULAS DO SALÁRIO
data['SALARY'] = data['SALARY'].astype(float) #TRANSFORMANDO O SALÁRIO EM FLOAT
data['AGE'] = data['AGE'].astype(int) #TRANSFORMANDO A IDADE EM INT

data.info()

In [ ]:
data.loc[data['NAME'] == 'Stephen Curry'] #LOCALIZAR O STEPHEN CURRY
target = data.loc[data['NAME'] == "Stephen Curry"].copy()
data.drop([103],inplace=True) #REMOVER O STEPHEN CURRY

In [ ]:
plt.figure(figsize=(16,8))
hm = sns.heatmap(data.corr(), annot=True, cmap="YlGnBu") #"YlGnBu"


In [ ]:
correlation = data.corr()
relevant_cols = []
salary_corr = correlation['SALARY'].to_dict()

for item in salary_corr.items():
    if item[1] > 0.3:
        relevant_cols.append(item[0])

relevant_cols

In [ ]:
delete_rows = []

for item in data.columns:
    if item not in relevant_cols:
        delete_rows.append(item)

data.drop(columns=delete_rows,inplace=True)
data

In [ ]:
X = data.drop(columns=['SALARY']).copy()
y = data['SALARY'].copy()   

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=80)
X_train.info()

In [ ]:
pipe = Pipeline([('scaler', StandardScaler())])

X_train_transformed = pipe.fit_transform(X_train)

plt.figure(figsize = (10, 6))


In [ ]:
linreg_model = LinearRegression()

scores = cross_val_score(linreg_model,X_train_transformed,y_train,scoring='neg_mean_squared_error',cv=4)
lin_reg_rmse_score=np.sqrt(-scores)

print(lin_reg_rmse_score)
print(lin_reg_rmse_score.mean())
print(lin_reg_rmse_score.std())

In [ ]:
tree_model = DecisionTreeRegressor()

scores=cross_val_score(tree_model,X_train_transformed,y_train,scoring='neg_mean_squared_error',cv=4)
dec_tree_reg_rmse_score=np.sqrt(-scores)

print(dec_tree_reg_rmse_score)
print(dec_tree_reg_rmse_score.mean())
print(dec_tree_reg_rmse_score.std())

In [ ]:
rforest_model=RandomForestRegressor()

scores=cross_val_score(rforest_model,X_train_transformed,y_train,scoring='neg_mean_squared_error',cv=4)
rand_forest_reg_rmse_score=np.sqrt(-scores)

print(rand_forest_reg_rmse_score)
print(rand_forest_reg_rmse_score.mean())
print(rand_forest_reg_rmse_score.std())

In [ ]:
linreg_model.fit(X_train_transformed,y_train)

X_test_transformed = pipe.fit_transform(X_test)

predicted_salary = linreg_model.predict(X_test_transformed)

linreg_model.score(X_test_transformed, y_test)

In [ ]:
rand_forest_reg_rmse = np.sqrt(mean_squared_error(y_test, predicted_salary))

print(rand_forest_reg_rmse)


In [ ]:
for item in delete_rows:
    target.drop(columns=item,inplace=True)

y_target = target['SALARY'].copy()
X_target = target.drop(columns=['SALARY']).copy()

X_target_transformed = pipe.fit_transform(X_target)
target_salary_predicted = linreg_model.predict(X_target_transformed)
print('Salário justo é: ' + 'US${0:.2f}'.format(target_salary_predicted[0]) + ' O salário real é: US$' + y_target.to_string(index=False))
